In [1]:
'''Import libraries'''
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW, Trainer, TrainingArguments
from tqdm import tqdm
from torch.nn import functional as F
import torch.nn as nn
import pickle

wandb.login()

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alberto-rodero557. Use `wandb login --relogin` to force relogin


True

In [2]:
'''Variables and parameters'''

SAMPLES_TO_TRAIN=1000
DIMENSIONS=200

N_LABELS=2
MAX_LEN = 256
EPOCHS=100
PATIENCE=10
LEARNING_RATE=.00005
WEIGHT_DECAY=.01
BATCH_SIZE=16
METRIC_FOR_BEST_MODEL='eval_loss'
if METRIC_FOR_BEST_MODEL=='eval_loss':
    GREATER_IS_BETTER = False
else:
    GREATER_IS_BETTER = True

In [3]:
'''Preparing dataset'''

file_path = 'datasets/subtaskA_glove_train_dev_monolingual.pkl'

# Load the data from the pickle file
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Extract the individual datasets from the loaded data
train_x = data['train_x']
train_y = data['train_y']
dev_x = data['dev_x']
dev_y = data['dev_y']

if SAMPLES_TO_TRAIN!=-1:
    random_indices = np.random.choice(train_x.shape[0], size=SAMPLES_TO_TRAIN, replace=False)
    train_x = train_x[random_indices]
    train_y = train_y[random_indices]

    random_indices_dev = np.random.choice(dev_x.shape[0], size=int(train_x.shape[0] * 0.2), replace=False)
    dev_x = dev_x[random_indices_dev]
    dev_y = pd.Series(dev_y.to_numpy()[random_indices_dev])

print(train_x.shape)
print(train_y.shape)
print(dev_x.shape)
print(dev_y.shape)

print(type(train_y))
print(type(dev_y))

(1000, 200)
(1000,)
(200, 200)
(200,)
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [4]:
'''metrics'''

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'auc': auc,
        'precision': precision,
        'recall': recall,
    }

In [5]:
class Data(Dataset):
    def __init__(self, X_train, y_train):
        self.X = torch.from_numpy(X_train.astype(np.float32))
        self.y = torch.from_numpy(y_train).type(torch.LongTensor)
        self.len = self.X.shape[0]

    def __getitem__(self, index):
        return {'input_ids': self.X[index], 'labels': self.y[index]}

    def __len__(self):
        return self.len

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y.values, test_size=0.2, random_state=42)
traindata = Data(X_train, y_train)
testdata = Data(X_test, y_test)

devdata=Data(dev_x, dev_y.values)

print(traindata)

In [6]:
# number of features (len of X cols)
input_dim = train_x.shape[-1]

# number of classes (unique of y)
output_dim = 2

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.linear1 = nn.Linear(input_dim, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.5)
        
        self.linear2 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.5)
        
        self.linear3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(0.5)
        
        self.linear4 = nn.Linear(256, output_dim)
        
        self.loss = nn.CrossEntropyLoss()

    def forward(self, input_ids, labels=None):
        x1 = F.leaky_relu(self.linear1(input_ids))
        x1 = self.bn1(x1)
        x1 = self.dropout1(x1)
        
        x2 = F.leaky_relu(self.linear2(x1))
        x2 = self.bn2(x2)
        x2 = self.dropout2(x2)
        
        # Adding the first skip connection
        x2 += x1
        
        x3 = F.leaky_relu(self.linear3(x2))
        x3 = self.bn3(x3)
        x3 = self.dropout3(x3)
        
        x4 = self.linear4(x3)
        
        outputs = (x4,)
        if labels is not None:
            loss = self.loss(x4, labels)
            outputs = (loss,) + outputs
            
        return (outputs if len(outputs) > 1 else outputs[0])

# Create the model
model = Network()

In [7]:
from transformers import EarlyStoppingCallback

model = Network()

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=500,
    weight_decay=WEIGHT_DECAY,
    metric_for_best_model=METRIC_FOR_BEST_MODEL,
    greater_is_better=GREATER_IS_BETTER,
    logging_dir='./logs',
    logging_steps=15000,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    push_to_hub=False,
    logging_first_step=False,
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="wandb"
)

# Create trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=traindata,             # training dataset
    eval_dataset=testdata, 
    compute_metrics=compute_metrics,# training dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)]
)

# Train the model
print(trainer.evaluate())

trainer.train()

print(trainer.evaluate())

  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:00<00:00, 19.44it/s]


{'eval_loss': 0.698131799697876, 'eval_accuracy': 0.435, 'eval_f1': 0.6062717770034843, 'eval_auc': 0.5, 'eval_precision': 0.435, 'eval_recall': 1.0, 'eval_runtime': 1.2849, 'eval_samples_per_second': 155.65, 'eval_steps_per_second': 10.117}


                                                  
  2%|▏         | 90/5000 [00:00<00:18, 271.23it/s]

{'eval_loss': 0.7775034308433533, 'eval_accuracy': 0.435, 'eval_f1': 0.6062717770034843, 'eval_auc': 0.5, 'eval_precision': 0.435, 'eval_recall': 1.0, 'eval_runtime': 0.0135, 'eval_samples_per_second': 14808.565, 'eval_steps_per_second': 962.557, 'epoch': 1.0}


...

{'eval_loss': 0.5278418064117432, 'eval_accuracy': 0.74, 'eval_f1': 0.6708860759493671, 'eval_auc': 0.7249516834503102, 'eval_precision': 0.7464788732394366, 'eval_recall': 0.6091954022988506, 'eval_runtime': 0.0175, 'eval_samples_per_second': 11408.105, 'eval_steps_per_second': 741.527, 'epoch': 73.0}


 73%|███████▎  | 3650/5000 [00:11<00:04, 326.15it/s]


{'train_runtime': 11.191, 'train_samples_per_second': 7148.588, 'train_steps_per_second': 446.787, 'train_loss': 0.5470019865689212, 'epoch': 73.0}


100%|██████████| 13/13 [00:00<00:00, 552.46it/s]

{'eval_loss': 0.5169082880020142, 'eval_accuracy': 0.745, 'eval_f1': 0.6832298136645962, 'eval_auc': 0.7320211575628115, 'eval_precision': 0.7432432432432432, 'eval_recall': 0.632183908045977, 'eval_runtime': 0.0265, 'eval_samples_per_second': 7538.221, 'eval_steps_per_second': 489.984, 'epoch': 73.0}


In [8]:
trainer.evaluate(devdata)

100%|██████████| 13/13 [00:00<00:00, 886.83it/s]


{'eval_loss': 1.4113526344299316,
 'eval_accuracy': 0.465,
 'eval_f1': 0.17054263565891473,
 'eval_auc': 0.45475928335501953,
 'eval_precision': 0.34375,
 'eval_recall': 0.1134020618556701,
 'eval_runtime': 0.0154,
 'eval_samples_per_second': 12977.627,
 'eval_steps_per_second': 843.546,
 'epoch': 73.0}

In [9]:
trainer.save_model('SavedModels/'+'dense_'+str(round(SAMPLES_TO_TRAIN/1000))+'k')

In [ ]:
# All samples trained with training dataset
print(''''eval_loss': 0.2924209237098694, 'eval_accuracy': 0.8761690046760187, 'eval_f1': 0.8671385056441498, 'eval_auc': 0.8748671641964536, 'eval_precision': 0.8905962458594038, 'eval_recall': 0.8448847765363129''')

'eval_loss': 0.2924209237098694, 'eval_accuracy': 0.8761690046760187, 'eval_f1': 0.8671385056441498, 'eval_auc': 0.8748671641964536, 'eval_precision': 0.8905962458594038, 'eval_recall': 0.8448847765363129
